In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_dir='/content/drive/MyDrive/one_class/train'
validation_dir='/content/drive/MyDrive/one_class/test'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip=True,vertical_flip=True,)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(240, 240),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(240, 240),
        batch_size=20,
        class_mode='binary')
print(train_generator.class_indices)

Found 676 images belonging to 2 classes.
Found 128 images belonging to 2 classes.
{'log': 0, 'nonlog': 1}


In [ ]:
feature_extractor_model = "https://tfhub.dev/google/efficientnet/b7/classification/1"


model = tf.keras.Sequential([
  hub.KerasLayer(feature_extractor_model, input_shape=(240, 240, 3), trainable=False),
  tf.keras.layers.Dense(1,activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])


In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=25,  
      epochs=20,
      validation_data=validation_generator,
      validation_steps=5,  
      verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
25/25 [==============================] - 384s 14s/step - loss: 0.3999 - acc: 0.8461 - val_loss: 0.1165 - val_acc: 0.9700
Epoch 2/20
25/25 [==============================] - 81s 3s/step - loss: 0.1228 - acc: 0.9647 - val_loss: 0.0791 - val_acc: 0.9800
Epoch 3/20
25/25 [==============================] - 30s 1s/step - loss: 0.0752 - acc: 0.9740 - val_loss: 0.0547 - val_acc: 0.9800
Epoch 4/20
25/25 [==============================] - 20s 794ms/step - loss: 0.0836 - acc: 0.9645 - val_loss: 0.0469 - val_acc: 0.9800
Epoch 5/20
25/25 [==============================] - 18s 740ms/step - loss: 0.0508 - acc: 0.9974 - val_loss: 0.0267 - val_acc: 1.0000
Epoch 6/20
25/25 [==============================] - 20s 790ms/step - loss: 0.0411 - acc: 0.9941 - val_loss: 0.0304 - val_acc: 1.0000
Epoch 7/20
25/25 [==============================] - 18s 721ms/step - loss: 0.0189 - acc: 1.0000 - val_loss: 0.0244 - val_acc: 1.0000
Epoch 8/20
25/25 [==============================] - 18s 728ms/step - loss: 0

In [ ]:
model.save('/content/drive/MyDrive/log_classifier1')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/log_classifier1/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/log_classifier1/assets


In [ ]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/log_classifier1')

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(240,240))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images)
  if classes[0]>0.5:
    print("not log")
  else:
    print("log")

Saving woodenlog_395.jpg to woodenlog_395.jpg
log
